In [ ]:
#NLP library ,for preprocessing of text
import nltk
#for numerical computations
import numpy as np 


In [ ]:
#to implement word embeddings ,word to vector or load the built model.

import gensim

In [ ]:
#Pre trained word2vec (word to vector) model 
#I downloaded bin file from https://ibm.ent.box.com/s/77etivy69jmga0x0u6vs2n47ul8baks4

model_pre_process=gensim.models.Word2Vec.load("E:\India_AI\word2vec.bin")

In [ ]:
#took very simple training data
#x=["Hi","How are you","I am also good","what are the stages in the comptition?","ok"]
#y=["Hi","I am doing well" ,"how about you","That's good to hear,so what do you want to know","well,one is idea submission round,second is Build","All the best"]

In [ ]:
import json 
sourcefile=open("E:\\India_AI\\conversation.json","rU")
conversation_data=json.load(sourcefile)
conversation=conversation_data["conversations"]
                
x=[]
y=[]
                
for i in range(len(conversation)):
    for j in range(len(conversation[i])):
        if j<len(conversation[i])-1:
            x.append(conversation[i][j])
            y.append(conversation[i][j+1])

In [ ]:
#Step 1 -Data Preprocessing -tokenize the sentence into words
tok_x_qa=[]
tok_y_qa=[]
for i in range(len(x)):
    tok_x_qa.append(nltk.word_tokenize(x[i].lower()))
    tok_y_qa.append(nltk.word_tokenize(y[i].lower()))
    

In [ ]:
#Step 2 -Data Preprocessing-Keep only those words which are avaiable in our trained word2vec model.
vec_x=[]
for tok in tok_x_qa:
    sentvec=[model_pre_process[w] for w in tok if w in model_pre_process.vocab]
    vec_x.append(sentvec)

In [ ]:
#Step 2 repeted for y as well
vec_y=[]
for tok in tok_y_qa:
    sentvec=[model_pre_process[w] for w in tok if w in model_pre_process.vocab]
    vec_y.append(sentvec)

In [ ]:
#took ones array of length 200 
#will use for converting variable length sequences to fixed length
sentpadding=np.ones((300),dtype=np.float32)

In [ ]:
sentpadding.shape

In [ ]:
#Curenly BOT can understand only till 14 characters
for tok_vec in vec_x:
    tok_vec[14:]=[]
    tok_vec.append(sentpadding)
    

In [ ]:
#Append the sentpadding if len is less than 14
for tok_vec in vec_x:
    if len(tok_vec)<15:
        for i in range(15-len(tok_vec)):
            tok_vec.append(sentpadding)
        
    

In [ ]:
#Same has done for y
for tok_vec in vec_y:
    if len(tok_vec)<15:
        for i in range(15-len(tok_vec)):
            tok_vec.append(sentpadding)
            
#Preprocessing End

In [ ]:
#Start Chatbot model 
#for splitting test and train data
from sklearn.model_selection import train_test_split
#to initate keras model
from keras.models import Sequential
#to implement LSTM and RNN
from keras.layers.recurrent import LSTM,SimpleRNN
import theano

In [ ]:
#to split vec_x and vec_y 
#np is numpy here

vec_x=np.array(vec_x,dtype=np.float64)
vec_y=np.array(vec_y,dtype=np.float64)

In [ ]:
#use train_test_split function to split training data 80% and test data 20% and its random to avoiad bias
x_train,x_test,y_train,y_test=train_test_split(vec_x,vec_y,test_size=0.1,random_state=1)

In [ ]:
#Initita the Layer in Recurrent Nueral Network
model=Sequential()

In [ ]:
#Adding 5 hidden layer in Neural network with required paraemeters
model.add(LSTM(output_dim=300,input_shape=x_train.shape[1:],return_sequences=True,init="glorot_normal",inner_init="glorot_normal",activation="sigmoid"))
model.add(LSTM(output_dim=300,input_shape=x_train.shape[1:],return_sequences=True,init="glorot_normal",inner_init="glorot_normal",activation="sigmoid"))
model.add(LSTM(output_dim=300,input_shape=x_train.shape[1:],return_sequences=True,init="glorot_normal",inner_init="glorot_normal",activation="sigmoid"))
model.add(LSTM(output_dim=300,input_shape=x_train.shape[1:],return_sequences=True,init="glorot_normal",inner_init="glorot_normal",activation="sigmoid"))
model.add(LSTM(output_dim=300,input_shape=x_train.shape[1:],return_sequences=True,init="glorot_normal",inner_init="glorot_normal",activation="sigmoid"))

In [ ]:
#Use cosine_proximity as word embedding works on cosine proximity and optimizer is gradient descendants .
model.compile(loss="cosine_proximity",optimizer="adam",metrics=["accuracy"])


In [ ]:
#Training the model ,can take more than epoch that can increase the accuracy and more computation required
model.fit(x_train,y_train,nb_epoch=2000,validation_data=(x_test,y_test))

In [ ]:
#Saving the chatbot 
#we will use this trained model on input data
model.save("chatbot_trained")

In [ ]:
predictions=model.predict(x_test)